In [26]:
# Algo-Trading System with ML & Automation (No TA-Lib Required)
# Complete implementation using pandas for technical indicators

import yfinance as yf
import pandas as pd
import numpy as np
import logging
from datetime import datetime, timedelta
import json
import os
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [27]:

# Google Sheets API (Optional - will handle missing gracefully)
try:
    import gspread
    from google.oauth2.service_account import Credentials
    SHEETS_AVAILABLE = True
except ImportError:
    SHEETS_AVAILABLE = False
    print("Google Sheets libraries not installed. Sheets integration disabled.")

# Telegram Bot (Optional)
try:
    import requests
    TELEGRAM_AVAILABLE = True
except ImportError:
    TELEGRAM_AVAILABLE = False
    print("Requests library not available. Telegram integration disabled.")


In [28]:
class TechnicalIndicators:
    """Calculate technical indicators using pandas (no TA-Lib required)"""
    
    @staticmethod
    def rsi(prices, period=14):
        """Calculate RSI using pandas"""
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    @staticmethod
    def sma(prices, period):
        """Simple Moving Average"""
        return prices.rolling(window=period).mean()
    
    @staticmethod
    def ema(prices, period):
        """Exponential Moving Average"""
        return prices.ewm(span=period).mean()
    
    @staticmethod
    def macd(prices, fast=12, slow=26, signal=9):
        """MACD calculation"""
        ema_fast = TechnicalIndicators.ema(prices, fast)
        ema_slow = TechnicalIndicators.ema(prices, slow)
        macd_line = ema_fast - ema_slow
        signal_line = TechnicalIndicators.ema(macd_line, signal)
        histogram = macd_line - signal_line
        return macd_line, signal_line, histogram
    
    @staticmethod
    def bollinger_bands(prices, period=20, std_dev=2):
        """Bollinger Bands calculation"""
        sma = TechnicalIndicators.sma(prices, period)
        std = prices.rolling(window=period).std()
        upper_band = sma + (std * std_dev)
        lower_band = sma - (std * std_dev)
        return upper_band, sma, lower_band

In [29]:

class DataIngestion:
    """Handle stock data fetching and preprocessing"""
    
    def __init__(self):
        self.logger = self._setup_logger()
        
    def _setup_logger(self):
        """Setup logging configuration"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('trading_system.log'),
                logging.StreamHandler()
            ]
        )
        return logging.getLogger(__name__)
    
    def fetch_nifty50_stocks(self, symbols: List[str], period: str = "1y") -> Dict[str, pd.DataFrame]:
        """
        Fetch stock data for NIFTY 50 stocks
        
        Args:
            symbols: List of stock symbols (e.g., ['RELIANCE.NS', 'TCS.NS', 'INFY.NS'])
            period: Time period for data (1y, 6mo, 3mo, etc.)
            
        Returns:
            Dictionary with stock data DataFrames
        """
        stock_data = {}
        
        for symbol in symbols:
            try:
                self.logger.info(f"Fetching data for {symbol}")
                ticker = yf.Ticker(symbol)
                data = ticker.history(period=period)
                
                if not data.empty:
                    stock_data[symbol] = data
                    self.logger.info(f"Successfully fetched {len(data)} records for {symbol}")
                else:
                    self.logger.warning(f"No data found for {symbol}")
                    
            except Exception as e:
                self.logger.error(f"Error fetching data for {symbol}: {str(e)}")
                
        return stock_data
    
    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate technical indicators using pandas
        
        Args:
            df: Stock price DataFrame
            
        Returns:
            DataFrame with technical indicators
        """
        df = df.copy()
        
        # RSI (14-period)
        df['RSI'] = TechnicalIndicators.rsi(df['Close'], 14)
        
        # Moving Averages
        df['MA_20'] = TechnicalIndicators.sma(df['Close'], 20)
        df['MA_50'] = TechnicalIndicators.sma(df['Close'], 50)
        df['EMA_12'] = TechnicalIndicators.ema(df['Close'], 12)
        df['EMA_26'] = TechnicalIndicators.ema(df['Close'], 26)
        
        # MACD
        macd, macd_signal, macd_hist = TechnicalIndicators.macd(df['Close'])
        df['MACD'] = macd
        df['MACD_Signal'] = macd_signal
        df['MACD_Hist'] = macd_hist
        
        # Bollinger Bands
        bb_upper, bb_middle, bb_lower = TechnicalIndicators.bollinger_bands(df['Close'])
        df['BB_Upper'] = bb_upper
        df['BB_Middle'] = bb_middle
        df['BB_Lower'] = bb_lower
        
        # Volume indicators
        df['Volume_MA'] = TechnicalIndicators.sma(df['Volume'], 20)
        df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']
        
        # Price change indicators
        df['Price_Change'] = df['Close'].pct_change()
        df['High_Low_Ratio'] = df['High'] / df['Low']
        
        return df

In [30]:

class TradingStrategy:
    """Implement RSI + Moving Average crossover strategy"""
    
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        
    def generate_signals(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Generate buy/sell signals based on RSI and MA crossover
        
        Strategy:
        - Buy: RSI < 30 AND MA_20 > MA_50
        - Sell: RSI > 70 OR MA_20 < MA_50
        
        Args:
            df: DataFrame with technical indicators
            
        Returns:
            DataFrame with signals
        """
        df = df.copy()
        
        # Initialize signals
        df['Signal'] = 0
        df['Position'] = 0
        
        # Buy signals: RSI < 30 and MA_20 > MA_50
        buy_condition = (df['RSI'] < 30) & (df['MA_20'] > df['MA_50'])
        
        # Sell signals: RSI > 70 or MA_20 < MA_50
        sell_condition = (df['RSI'] > 70) | (df['MA_20'] < df['MA_50'])
        
        df.loc[buy_condition, 'Signal'] = 1  # Buy
        df.loc[sell_condition, 'Signal'] = -1  # Sell
        
        # Generate positions (forward fill)
        df['Position'] = df['Signal'].replace(0, np.nan).fillna(method='ffill').fillna(0)
        
        return df
    
    def backtest_strategy(self, df: pd.DataFrame, initial_capital: float = 100000) -> Dict:
        """
        Backtest the trading strategy
        
        Args:
            df: DataFrame with signals
            initial_capital: Starting capital
            
        Returns:
            Backtest results dictionary
        """
        df = df.copy()
        
        # Calculate returns
        df['Returns'] = df['Close'].pct_change()
        df['Strategy_Returns'] = df['Position'].shift(1) * df['Returns']
        
        # Calculate cumulative returns
        df['Cumulative_Returns'] = (1 + df['Returns']).cumprod()
        df['Cumulative_Strategy_Returns'] = (1 + df['Strategy_Returns']).cumprod()
        
        # Calculate portfolio value
        df['Portfolio_Value'] = initial_capital * df['Cumulative_Strategy_Returns']
        
        # Generate trade log
        trades = []
        position = 0
        entry_price = 0
        entry_date = None
        
        for idx, row in df.iterrows():
            if row['Signal'] == 1 and position == 0:  # Buy signal
                position = 1
                entry_price = row['Close']
                entry_date = idx
                
            elif row['Signal'] == -1 and position == 1:  # Sell signal
                exit_price = row['Close']
                exit_date = idx
                pnl = (exit_price - entry_price) / entry_price * 100
                
                trades.append({
                    'Entry_Date': entry_date,
                    'Exit_Date': exit_date,
                    'Entry_Price': entry_price,
                    'Exit_Price': exit_price,
                    'PnL_Percent': pnl,
                    'PnL_Amount': (exit_price - entry_price) * (initial_capital / entry_price)
                })
                
                position = 0
        
        # Calculate performance metrics
        final_value = df['Portfolio_Value'].iloc[-1] if not df['Portfolio_Value'].empty else initial_capital
        total_return = (final_value / initial_capital - 1) * 100
        buy_hold_return = (df['Cumulative_Returns'].iloc[-1] - 1) * 100 if not df['Cumulative_Returns'].empty else 0
        
        win_trades = [t for t in trades if t['PnL_Percent'] > 0]
        lose_trades = [t for t in trades if t['PnL_Percent'] <= 0]
        
        win_ratio = len(win_trades) / len(trades) * 100 if trades else 0
        
        avg_win = np.mean([t['PnL_Percent'] for t in win_trades]) if win_trades else 0
        avg_loss = np.mean([t['PnL_Percent'] for t in lose_trades]) if lose_trades else 0
        
        results = {
            'total_trades': len(trades),
            'winning_trades': len(win_trades),
            'losing_trades': len(lose_trades),
            'win_ratio': win_ratio,
            'total_return': total_return,
            'buy_hold_return': buy_hold_return,
            'avg_win': avg_win,
            'avg_loss': avg_loss,
            'final_portfolio_value': final_value,
            'trades': trades,
            'df_with_signals': df
        }
        
        return results

In [31]:
class MLPredictor:
    """Machine Learning component for price prediction"""
    
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self.scaler = StandardScaler()
        self.model = None
        
    def prepare_features(self, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Prepare features for ML model
        
        Args:
            df: DataFrame with technical indicators
            
        Returns:
            Features array and target array
        """
        df = df.copy()
        
        # Create target variable (next day price movement)
        df['Next_Return'] = df['Close'].shift(-1) / df['Close'] - 1
        df['Target'] = (df['Next_Return'] > 0).astype(int)  # 1 for up, 0 for down
        
        # Select features
        feature_cols = ['RSI', 'MACD', 'MACD_Signal', 'Volume_Ratio', 'Price_Change', 'High_Low_Ratio']
        
        # Remove rows with NaN values
        df_clean = df[feature_cols + ['Target']].dropna()
        
        if len(df_clean) == 0:
            return np.array([]), np.array([])
        
        X = df_clean[feature_cols].values
        y = df_clean['Target'].values
        
        return X, y
    
    def train_model(self, X: np.ndarray, y: np.ndarray, model_type: str = 'decision_tree') -> Dict:
        """
        Train ML model for price prediction
        
        Args:
            X: Features array
            y: Target array
            model_type: Type of model ('decision_tree' or 'logistic_regression')
            
        Returns:
            Training results dictionary
        """
        if len(X) == 0 or len(y) == 0:
            return {'error': 'No data available for training'}
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y if len(np.unique(y)) > 1 else None
        )
        
        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        # Train model
        if model_type == 'decision_tree':
            self.model = DecisionTreeClassifier(random_state=42, max_depth=10)
        else:
            self.model = LogisticRegression(random_state=42, max_iter=1000)
            
        self.model.fit(X_train_scaled, y_train)
        
        # Make predictions
        y_pred = self.model.predict(X_test_scaled)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        
        results = {
            'model_type': model_type,
            'accuracy': accuracy,
            'classification_report': classification_report(y_test, y_pred),
            'feature_importance': getattr(self.model, 'feature_importances_', None)
        }
        
        self.logger.info(f"Model trained with accuracy: {accuracy:.4f}")
        
        return results
    
    def predict_next_movement(self, features: np.ndarray) -> Tuple[int, float]:
        """
        Predict next day price movement
        
        Args:
            features: Feature array for prediction
            
        Returns:
            Prediction (0/1) and probability
        """
        if self.model is None:
            raise ValueError("Model not trained yet")
            
        features_scaled = self.scaler.transform(features.reshape(1, -1))
        prediction = self.model.predict(features_scaled)[0]
        probability = self.model.predict_proba(features_scaled)[0].max()
        
        return prediction, probability

In [32]:
class AlgoTradingSystem:
    """Main algo-trading system orchestrator"""
    
    def __init__(self, config: Dict = None):
        self.config = config or {}
        self.logger = self._setup_logger()
        
        # Initialize components
        self.data_ingestion = DataIngestion()
        self.trading_strategy = TradingStrategy()
        self.ml_predictor = MLPredictor()
    
    def _setup_logger(self):
        """Setup logging configuration"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('trading_system.log'),
                logging.StreamHandler()
            ]
        )
        return logging.getLogger(__name__)
    
    def run_analysis(self, symbols: List[str]):
        """Run complete analysis pipeline"""
        self.logger.info("Starting algo-trading analysis...")
        
        # 1. Data Ingestion
        self.logger.info("Step 1: Fetching stock data...")
        stock_data = self.data_ingestion.fetch_nifty50_stocks(symbols, period="1y")
        
        results = {}
        
        for symbol, df in stock_data.items():
            self.logger.info(f"Processing {symbol}...")
            
            try:
                # 2. Calculate technical indicators
                df_with_indicators = self.data_ingestion.calculate_technical_indicators(df)
                
                # 3. Generate trading signals
                df_with_signals = self.trading_strategy.generate_signals(df_with_indicators)
                
                # 4. Backtest strategy
                backtest_results = self.trading_strategy.backtest_strategy(df_with_signals)
                
                # 5. ML Prediction (if enabled)
                ml_results = None
                if self.config.get('enable_ml', True):
                    try:
                        X, y = self.ml_predictor.prepare_features(df_with_indicators)
                        if len(X) > 50:  # Minimum data requirement
                            ml_results = self.ml_predictor.train_model(X, y)
                            
                            # Make prediction for latest data
                            if not 'error' in ml_results:
                                latest_features = X[-1]
                                prediction, probability = self.ml_predictor.predict_next_movement(latest_features)
                                ml_results['latest_prediction'] = {
                                    'prediction': prediction,
                                    'probability': probability
                                }
                    except Exception as e:
                        self.logger.error(f"ML prediction failed for {symbol}: {str(e)}")
                        ml_results = {'error': str(e)}
                
                results[symbol] = {
                    'backtest_results': backtest_results,
                    'ml_results': ml_results,
                    'latest_data': df_with_signals.tail(1).to_dict('records')[0] if not df_with_signals.empty else {}
                }
                
                self.logger.info(f"Completed analysis for {symbol}")
                
            except Exception as e:
                self.logger.error(f"Error processing {symbol}: {str(e)}")
                results[symbol] = {'error': str(e)}
        
        return results
    
    def print_results_summary(self, results: Dict):
        """Print analysis results summary"""
        print("\n" + "="*80)
        print("ALGO-TRADING SYSTEM RESULTS SUMMARY")
        print("="*80)
        
        for symbol, result in results.items():
            print(f"\n{symbol}")
            print("-" * 50)
            
            if 'error' in result:
                print(f"Error: {result['error']}")
                continue
                
            backtest = result['backtest_results']
            print(f"Total Trades: {backtest['total_trades']}")
            print(f"Win Ratio: {backtest['win_ratio']:.2f}%")
            print(f"Total Return: {backtest['total_return']:.2f}%")
            print(f"Buy & Hold Return: {backtest['buy_hold_return']:.2f}%")
            print(f"Final Portfolio Value: ₹{backtest['final_portfolio_value']:,.2f}")
            
            if result['ml_results'] and 'error' not in result['ml_results']:
                ml = result['ml_results']
                print(f"ML Model Accuracy: {ml['accuracy']:.4f}")
                if 'latest_prediction' in ml:
                    pred = ml['latest_prediction']
                    direction = "UP ⬆️" if pred['prediction'] == 1 else "DOWN ⬇️"
                    print(f"Next Day Prediction: {direction} ({pred['probability']:.2f})")
            
            if result['latest_data']:
                latest = result['latest_data']
                rsi_val = latest.get('RSI', 0)
                signal_val = latest.get('Signal', 0)
                print(f"Current RSI: {rsi_val:.2f}")
                print(f"Current Signal: {'BUY 🟢' if signal_val == 1 else 'SELL 🔴' if signal_val == -1 else 'HOLD 🟡'}")


In [33]:
def quick_demo():
    
    print("ALGO-TRADING SYSTEM DEMO")
    print("=" * 50)
    
    # Configuration
    config = {'enable_ml': True}
    
    # NIFTY 50 stocks (top 3 for demo)
    symbols = [
        'RELIANCE.NS',  # Reliance Industries
        'TCS.NS',       # Tata Consultancy Services
        'INFY.NS'       # Infosys
    ]
    
    # Initialize and run system
    trading_system = AlgoTradingSystem(config)
    
    try:
        results = trading_system.run_analysis(symbols)
        trading_system.print_results_summary(results)
        
        # Save results to JSON for further analysis
        with open('trading_results.json', 'w') as f:
            # Convert non-serializable objects for JSON
            json_results = {}
            for symbol, result in results.items():
                if 'error' not in result:
                    json_results[symbol] = {
                        'total_trades': result['backtest_results']['total_trades'],
                        'win_ratio': result['backtest_results']['win_ratio'],
                        'total_return': result['backtest_results']['total_return'],
                        'ml_accuracy': result['ml_results']['accuracy'] if result['ml_results'] and 'error' not in result['ml_results'] else None
                    }
                else:
                    json_results[symbol] = result
                    
            json.dump(json_results, f, indent=2)
        
        print(f"\n✅ Analysis complete! Results saved to trading_results.json")
        print(f"📊 Detailed logs available in trading_system.log")
        
        # Sample Google Sheets format output
        print(f"\n📊 Sample Google Sheets Data Format:")
        print("=" * 50)
        print("Trade_Log Sheet:")
        print("Symbol | Entry_Date | Exit_Date | Entry_Price | Exit_Price | PnL%")
        for symbol, result in results.items():
            if 'error' not in result and result['backtest_results']['trades']:
                trade = result['backtest_results']['trades'][0]  # First trade
                print(f"{symbol} | {trade['Entry_Date'].strftime('%Y-%m-%d')} | {trade['Exit_Date'].strftime('%Y-%m-%d')} | ₹{trade['Entry_Price']:.2f} | ₹{trade['Exit_Price']:.2f} | {trade['PnL_Percent']:.2f}%")
        
        return results
        
    except Exception as e:
        logging.error(f"System execution failed: {str(e)}")
        raise

In [34]:
if __name__ == "__main__":
    quick_demo()

2025-08-05 19:56:06,660 - __main__ - INFO - Starting algo-trading analysis...
2025-08-05 19:56:06,663 - __main__ - INFO - Step 1: Fetching stock data...
2025-08-05 19:56:06,663 - __main__ - INFO - Fetching data for RELIANCE.NS


ALGO-TRADING SYSTEM DEMO


2025-08-05 19:56:07,485 - __main__ - INFO - Successfully fetched 251 records for RELIANCE.NS
2025-08-05 19:56:07,485 - __main__ - INFO - Fetching data for TCS.NS
2025-08-05 19:56:07,802 - __main__ - INFO - Successfully fetched 251 records for TCS.NS
2025-08-05 19:56:07,802 - __main__ - INFO - Fetching data for INFY.NS
2025-08-05 19:56:08,514 - __main__ - INFO - Successfully fetched 251 records for INFY.NS
2025-08-05 19:56:08,514 - __main__ - INFO - Processing RELIANCE.NS...
2025-08-05 19:56:08,575 - __main__ - INFO - Model trained with accuracy: 0.4255
2025-08-05 19:56:08,583 - __main__ - INFO - Completed analysis for RELIANCE.NS
2025-08-05 19:56:08,584 - __main__ - INFO - Processing TCS.NS...
2025-08-05 19:56:08,649 - __main__ - INFO - Model trained with accuracy: 0.5319
2025-08-05 19:56:08,654 - __main__ - INFO - Completed analysis for TCS.NS
2025-08-05 19:56:08,655 - __main__ - INFO - Processing INFY.NS...
2025-08-05 19:56:08,725 - __main__ - INFO - Model trained with accuracy: 0.53


ALGO-TRADING SYSTEM RESULTS SUMMARY

RELIANCE.NS
--------------------------------------------------
Total Trades: 1
Win Ratio: 0.00%
Total Return: -1.61%
Buy & Hold Return: -3.52%
Final Portfolio Value: ₹98,391.32
ML Model Accuracy: 0.4255
Next Day Prediction: UP ⬆️ (1.00)
Current RSI: 28.57
Current Signal: SELL 🔴

TCS.NS
--------------------------------------------------
Total Trades: 2
Win Ratio: 50.00%
Total Return: 34.37%
Buy & Hold Return: -24.99%
Final Portfolio Value: ₹134,368.13
ML Model Accuracy: 0.5319
Next Day Prediction: UP ⬆️ (1.00)
Current RSI: 26.09
Current Signal: SELL 🔴

INFY.NS
--------------------------------------------------
Total Trades: 2
Win Ratio: 0.00%
Total Return: 10.02%
Buy & Hold Return: -14.51%
Final Portfolio Value: ₹110,015.19
ML Model Accuracy: 0.5319
Next Day Prediction: UP ⬆️ (1.00)
Current RSI: 11.76
Current Signal: SELL 🔴

✅ Analysis complete! Results saved to trading_results.json
📊 Detailed logs available in trading_system.log

📊 Sample Google Sh

In [39]:
###telegram alert system###
import logging
import platform
import os

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - ALERT - %(message)s')

# Beep sound function
def beep_alert():
    if platform.system() == "Windows":
        import winsound
        winsound.Beep(1000, 300)
    elif platform.system() == "Darwin":
        os.system('say "Alert"')
    else:
        print('\a')  # Linux/other

# Results summary from your trading output
results = {
    "RELIANCE.NS": {
        "accuracy": 0.4255,
        "win_ratio": 0.00,
        "total_return": -1.61,
        "signal": "SELL 🔴",
        "rsi": 28.57
    },
    "TCS.NS": {
        "accuracy": 0.5319,
        "win_ratio": 0.50,
        "total_return": 34.37,
        "signal": "SELL 🔴",
        "rsi": 26.09
    },
    "INFY.NS": {
        "accuracy": 0.5319,
        "win_ratio": 0.00,
        "total_return": 10.02,
        "signal": "SELL 🔴",
        "rsi": 11.76
    }
}

# Alert checker
def check_alerts(symbol, data):
    if data["accuracy"] < 0.5:
        logging.info(f"{symbol} - Low Model Accuracy: {data['accuracy']*100:.2f}%")
        beep_alert()
    if data["win_ratio"] == 0:
        logging.info(f"{symbol} - 0% Win Ratio! Strategy may need review.")
        beep_alert()
    if data["total_return"] < 0:
        logging.info(f"{symbol} - Negative Total Return: {data['total_return']}%")
        beep_alert()
    if data["signal"] == "SELL 🔴" and data["rsi"] < 30:
        logging.info(f"{symbol} - SELL Signal with RSI {data['rsi']:.2f} (Possibly Oversold)")
        beep_alert()

# Run alerts
for symbol, data in results.items():
    check_alerts(symbol, data)


2025-08-05 22:15:44,006 - root - INFO - RELIANCE.NS - Low Model Accuracy: 42.55%
2025-08-05 22:15:44,313 - root - INFO - RELIANCE.NS - 0% Win Ratio! Strategy may need review.
2025-08-05 22:15:44,629 - root - INFO - RELIANCE.NS - Negative Total Return: -1.61%
2025-08-05 22:15:44,936 - root - INFO - RELIANCE.NS - SELL Signal with RSI 28.57 (Possibly Oversold)
2025-08-05 22:15:45,238 - root - INFO - TCS.NS - SELL Signal with RSI 26.09 (Possibly Oversold)
2025-08-05 22:15:45,541 - root - INFO - INFY.NS - 0% Win Ratio! Strategy may need review.
2025-08-05 22:15:45,852 - root - INFO - INFY.NS - SELL Signal with RSI 11.76 (Possibly Oversold)
